In [1]:
!nvidia-smi

Tue Apr 19 02:18:22 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 15.4 MB/s 
     |████████████████████████████████| 895 kB 18.8 MB/s 
     |████████████████████████████████| 77 kB 7.1 MB/s 
     |████████████████████████████████| 596 kB 39.0 MB/s 
     |████████████████████████████████| 6.6 MB 39.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Read the following link to get kaggle.json https://www.kaggle.com/general/74235


In [4]:
import json
content={"username":"namlv1997","key":"5ecc995bb891689ee7f58417d65cbe37"}
with open('kaggle.json','w')as f:
  json.dump(content,f)

In [5]:
%%shell
mkdir ~/.kaggle
cp kaggle.json ~/.kaggle/
chmod 600 ~/.kaggle/kaggle.json

In [6]:
!kaggle competitions download -c fake-news

 80% 37.0M/46.5M [00:00<00:00, 104MB/s] 
100% 46.5M/46.5M [00:00<00:00, 111MB/s]


In [7]:
!unzip /content/fake-news.zip

Archive:  /content/fake-news.zip
  inflating: submit.csv              
  inflating: test.csv                
  inflating: train.csv               


In [68]:
from tqdm.auto import tqdm
import pandas as pd
import tensorflow as tf
import numpy as np
import string
import re
from os.path import join
import os
from transformers import AutoTokenizer
from transformers import TFAutoModelForSequenceClassification
from transformers import create_optimizer
from sklearn.metrics import f1_score
from transformers import AutoConfig
from itertools import chain
from collections import Counter
from torch.utils.data import DataLoader
from transformers import AutoModelForSequenceClassification
import torch
from torch import nn
from torch.utils.data import Dataset
from transformers import get_scheduler
from sklearn.metrics import classification_report,confusion_matrix

In [9]:
fake_news_folder="fake_news"
fake_new_dataset="raw_dataset"
fake_new_models="models"
name="bert-base-uncased"

In [10]:
root_path="/content/drive/MyDrive"
fake_news_folder_path=join(root_path,fake_news_folder)
fake_news_dataset_path=join(root_path,fake_news_folder,fake_new_dataset)
fake_news_models_path=join(root_path,fake_news_folder,fake_new_models)
model_path=f'{fake_news_models_path}/{name}/'

In [11]:
os.makedirs(model_path, exist_ok=True)
os.makedirs(fake_news_dataset_path, exist_ok=True)

# Load fake news dataset from kaggle

In [12]:
df=pd.read_csv("/content/train.csv",index_col=False)
df=df.drop(['id','title','author'],axis=1)
df=df.sample(frac=1)
df

,text,label
17256,Support Us The Illuminati Documentary SKULL AN...,1
20096,"(Before It's News)\nRemember when the media, B...",1
20299,"Nearly 150, 000 American teenagers from 13 to ...",0
3315,Who Paid Hillary Clinton $21.7 Million to Give...,1
2213,Radio host Rush Limbaugh discussed the riots a...,0
...,...,...
17931,WASHINGTON — President Donald Trump nominat...,0
12933,"\n \n\nN.Morgan )Â Project Veritas, the brain ...",1
19759,10 Questions For Every Atheist By Jonathan Len...,1
19107,SWEDEN HELP WANTED: Activities Coordinator for...,1


Split raw dataset to train set and validation set

In [13]:
train=df.iloc[:-2600]
valid=df.iloc[-2600:]

In [14]:
len(train),len(valid)

(18200, 2600)

Check data whether it is balanced or not

In [15]:
for i in range(2):
  samples=train[train['label']==i]
  print(f'label: {i}\thas {len(samples)} samples')

label: 0	has 9025 samples
label: 1	has 9175 samples


In [16]:
for i in range(2):
  samples=valid[valid['label']==i]
  print(f'label: {i}\thas {len(samples)} samples')

label: 0	has 1362 samples
label: 1	has 1238 samples


In [17]:
train.to_csv(join(fake_news_dataset_path,"train.csv"),index=False)

In [18]:
valid.to_csv(join(fake_news_dataset_path,"valid.csv"),index=False)

# Clean and preprocessing

##Basic clean data

In [19]:
train=pd.read_csv(join(fake_news_dataset_path,"train.csv"),index_col=False)
train=train.dropna(subset=['label', 'text'])

valid=pd.read_csv(join(fake_news_dataset_path,"valid.csv"),index_col=False)
valid=valid.dropna(subset=['label', 'text'])


In [42]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",

                           "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",

                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",

                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",

                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",

                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",

                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",

                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",

                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",

                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",

                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",

                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",

                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",

                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",

                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",

                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",

                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",

                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",

                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",

                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",

                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",

                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",

                           "you're": "you are", "you've": "you have"}

In [43]:
def white_space_fix(text):
    return ' '.join(text.split())

In [44]:
def normalize(s):
  def lower(text):
    return text.lower()
  
  def norm_apostrophe(text):
    return re.sub("’(\s|)","'",text)

  def replace_contraction(text):
    for k,v in contraction_mapping.items():
      text=text.replace(k,v)
    return text
  def remove_punc(text):
    return re.sub("\W"," ",text)

  def remove_articles(text):
    regex = re.compile(r'\b(a|an|the)\b', re.UNICODE)
    return re.sub(regex, ' ', text)

  return white_space_fix(remove_articles(remove_punc(replace_contraction((norm_apostrophe(lower(s))))))).strip()


In [24]:
train['text']=train['text'].map(normalize)

In [25]:
def find_top_common_tokens(list_texts,top_n=30):
  tokens=list(chain.from_iterable([i.split() for i in list_texts]))
  counter=Counter(tokens)
  top_common_tokens=dict(counter.most_common(top_n))
  return top_common_tokens

In [26]:
top_common_tokens=find_top_common_tokens(train['text'].tolist())

In [27]:
def remove_top_common_tokens(text):
  for k in top_common_tokens:
    text=text.replace(k," ")
  return white_space_fix(text).strip()

In [28]:
train['text']=train['text'].map(remove_top_common_tokens)

In [29]:
valid['text']=valid['text'].map(normalize)

In [30]:
# top_common_tokens=find_top_common_tokens(valid['text'].tolist())

In [31]:
# valid['text']=valid['text'].map(remove_top_common_tokens)

In [32]:
train=train[train['text']!=""]
valid=valid[valid['text']!=""]

In [33]:
len(train),len(valid)

(18092, 2591)

In [34]:
train.to_csv(join(model_path,"train.csv"),index=False)
valid.to_csv(join(model_path,"valid.csv"),index=False)

##Preprocessing

In [12]:
train=pd.read_csv(join(model_path,"train.csv"),index_col=False)
valid=pd.read_csv(join(model_path,"valid.csv"),index_col=False)

In [13]:
tokenizer = AutoTokenizer.from_pretrained(name)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [14]:
class FakeNewsDataset(Dataset):
    def __init__(self, input_ids, attention_mask,token_type_ids,labels,device):
        self.labels = labels
        self.input_ids=input_ids
        self.attention_mask=attention_mask
        self.device=device
        
    def __len__(self):
            return len(self.labels)
    def __getitem__(self, idx):
        label = self.labels[idx]
        input_ids = self.input_ids[idx]
        attention_mask = self.attention_mask[idx]
        sample = {"input_ids": input_ids.to(device),'attention_mask':attention_mask.to(device), "labels": label.to(device)}
        return sample

In [15]:
max_length=512
return_tensors='pt'

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [17]:
x_train=tokenizer(train['text'].tolist() ,padding=True,truncation=True,max_length=max_length,return_tensors=return_tensors)

In [18]:
y_train=torch.Tensor(train['label']).to(torch.long)

In [19]:
train_dataset={i:x_train[i] for i in x_train}
train_dataset.update({"labels":y_train})

In [20]:
train_dataset=FakeNewsDataset(**train_dataset,device=device)

In [21]:
train_batch_size=8
num_training_samples=len(y_train)
num_training_batches=int((num_training_samples/train_batch_size)+0.5)

In [22]:
train_dataset=DataLoader(train_dataset,batch_size=train_batch_size,drop_last=True)

In [23]:
x_valid=tokenizer(valid['text'].tolist() ,padding=True,truncation=True,max_length=max_length,return_tensors=return_tensors)#.to(device)

In [24]:
y_valid=torch.Tensor(valid['label']).to(torch.long)#.to(device)

In [25]:
valid_dataset={i:x_valid[i] for i in x_valid}
valid_dataset.update({"labels":y_valid})

In [26]:
valid_dataset=FakeNewsDataset(**valid_dataset,device=device)

In [27]:
valid_batch_size=8
num_valid_samples=len(y_valid)
num_valid_batches=int((num_valid_samples/valid_batch_size)+0.5)

In [28]:
valid_dataset=DataLoader(valid_dataset,batch_size=valid_batch_size,drop_last=True)

# Train

In [29]:
epochs=5
num_training_steps=epochs * len(train_dataset)
num_warmup_steps=600
lr=5e-5

Load the pretrained model

In [30]:
num_labels=len(train['label'].unique())
config = AutoConfig.from_pretrained(name)
config.num_labels=num_labels
model = AutoModelForSequenceClassification.from_pretrained(name,config=config).to(device)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [31]:
optimizer=torch.optim.Adam(model.parameters(),lr=lr)

In [32]:
lr_scheduler=get_scheduler(name='linear',optimizer=optimizer,num_warmup_steps =num_warmup_steps ,num_training_steps =num_training_steps)

In [33]:
def save_model(epoch,model,optimizer,loss,path):
  torch.save({
          'epoch': epoch,
          'model_state_dict': model.state_dict(),
          'optimizer_state_dict': optimizer.state_dict(),
          'loss': loss,
          }, join(path,f'ckpt{epoch}.pt'))

In [34]:
def train(optimizer,dataloader,lr_scheduler):
  model.train(True)
  loop = tqdm(dataloader)
  
  loss=0
  for batch in dataloader:
    model.zero_grad(set_to_none=True)
    outputs=model(**batch)

    outputs.loss.backward()
    _loss=outputs.loss

    loss+=_loss.item()
    optimizer.step()
    lr_scheduler.step()
    loop.set_postfix({f'train_loss_per_batch':_loss.item()})
    loop.update()
    
  return loss/len(dataloader)

In [35]:
def valid(dataloader):
  loop = tqdm(dataloader)
  model.eval()
  loss=0
  with torch.no_grad():
    for batch in dataloader:
      outputs=model(**batch)
      _loss=outputs.loss

      loss+=_loss.item()
      loop.set_postfix({f'valid_loss_per_batch':_loss.item()})
      loop.update()
  return loss/len(dataloader)

In [36]:
for epoch in range(epochs):
  train_loss=train(optimizer,train_dataset,lr_scheduler)
  valid_loss=valid(valid_dataset)
  print(f'epoch: {epoch} train_loss: {train_loss} valid_loss: {valid_loss}')
  save_model(epoch,model,optimizer,valid_loss,model_path)

  0%|          | 0/2261 [00:00<?, ?it/s]

  0%|          | 0/323 [00:00<?, ?it/s]

epoch: 0 train_loss: 0.300271340951485 valid_loss: 0.31349032320335923


  0%|          | 0/2261 [00:00<?, ?it/s]

  0%|          | 0/323 [00:00<?, ?it/s]

...

  0%|          | 0/323 [00:00<?, ?it/s]

epoch: 3 train_loss: 0.0221198821193716 valid_loss: 0.30651484896867154


  0%|          | 0/2261 [00:00<?, ?it/s]

  0%|          | 0/323 [00:00<?, ?it/s]

epoch: 4 train_loss: 0.008499613277190263 valid_loss: 0.36259793575005045


Load test set

In [37]:
test=pd.read_csv("/content/test.csv")

In [41]:
submit=pd.read_csv("/content/submit.csv")
labels=torch.Tensor(submit['label'])

In [45]:
test['text']=test['text'].astype(str).map(normalize)

In [46]:
samples=test['text'].tolist()

In [47]:
num_labels=len(submit['label'].unique())
num_labels

2

Load model

In [48]:
from transformers import AutoConfig
config = AutoConfig.from_pretrained(name)
config.num_labels=num_labels

In [59]:
model=AutoModelForSequenceClassification.from_pretrained(name,config=config).to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [60]:
model.num_labels

2

In [61]:
max_length=512
return_tensors='pt'
tokenizer = AutoTokenizer.from_pretrained(name)

In [83]:
checkpoint = torch.load("/content/drive/MyDrive/fake_news/models/bert-base-uncased/ckpt1.pt")
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

Load the weights

In [84]:
model.eval()
y_pred=[]
with torch.no_grad():
  loop = tqdm(range(len(samples)))
  for idx,sample in enumerate(samples):
    inputs=tokenizer(sample,padding=True,truncation=True,max_length=max_length,return_tensors=return_tensors).to(device)
    outputs=model(**inputs)
    prob=torch.nn.functional.softmax(outputs.logits,dim=1)
    preds=torch.argmax(prob,dim=1)
    y_pred.append(preds)
    loop.update()


  0%|          | 0/5200 [00:00<?, ?it/s]

In [85]:
y_pred=torch.Tensor(y_pred)

In [86]:
y_pred

tensor([0., 1., 1.,  ..., 0., 1., 0.])

In [87]:
labels

tensor([0., 1., 0.,  ..., 0., 1., 0.])

In [82]:
#0
f1_score(labels,y_pred,average='micro')

0.6411538461538462

In [88]:
#1
f1_score(labels,y_pred,average='micro')

0.6494230769230769

In [76]:
#3
f1_score(labels,y_pred,average='micro')

0.6501923076923077

In [67]:
#4
f1_score(labels,y_pred,average='micro')

0.6434615384615384

In [69]:
print(classification_report(labels,y_pred,digits=5))

              precision    recall  f1-score   support

         0.0    0.58670   0.70158   0.63902      2339
         1.0    0.70953   0.59595   0.64780      2861

    accuracy                        0.64346      5200
   macro avg    0.64811   0.64876   0.64341      5200
weighted avg    0.65428   0.64346   0.64385      5200



In [70]:
confusion_matrix(labels,y_pred)

array([[1641,  698],
       [1156, 1705]])